In [1]:
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from scipy import ndimage
from skimage.measure import regionprops
from skimage import io
from skimage.filters import threshold_otsu   # For finding the threshold for grayscale to binary conversion
import tensorflow as tf
import pandas as pd
import numpy as np
from time import time
import keras
from tensorflow.python.framework import ops
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior() 



D:\ANACONDA\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
D:\ANACONDA\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
D:\ANACONDA\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
D:\ANACONDA\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,



Instructions for updating:
non-resource variables are not supported in the long term


## Path defined

In [2]:
genuine_image_paths = r"D:\ML_Practice\Signature-Forgery-Detection\real"
forged_image_paths = r"D:\ML_Practice\Signature-Forgery-Detection\forged"

## Preprocessing the image

In [3]:
def rgbgrey(img):
    # Converts rgb to grayscale
    greyimg = np.zeros((img.shape[0], img.shape[1]))
    for row in range(len(img)):
        for col in range(len(img[row])):
            greyimg[row][col] = np.average(img[row][col])
    return greyimg

In [4]:
def greybin(img):
    # Converts grayscale to binary
    blur_radius = 0.8
    img = ndimage.gaussian_filter(img, blur_radius)  # to remove small components or noise
#     img = ndimage.binary_erosion(img).astype(img.dtype)
    thres = threshold_otsu(img)
    binimg = img > thres
    binimg = np.logical_not(binimg)
    return binimg

In [5]:
def preproc(path, img=None, display=True):
    if img is None:
        img = mpimg.imread(path)
    if display:
        plt.imshow(img)
        plt.show()
    grey = rgbgrey(img) #rgb to grey
    if display:
        plt.imshow(grey, cmap = matplotlib.cm.Greys_r)
        plt.show()
    binimg = greybin(grey) #grey to binary
    if display:
        plt.imshow(binimg, cmap = matplotlib.cm.Greys_r)
        plt.show()
    r, c = np.where(binimg==1)
    # Now we will make a bounding box with the boundary as the position of pixels on extreme.
    # Thus we will get a cropped image with only the signature part.
    signimg = binimg[r.min(): r.max(), c.min(): c.max()]
    if display:
        plt.imshow(signimg, cmap = matplotlib.cm.Greys_r)
        plt.show()
    return signimg

## Feature Extraction


In [6]:
def Ratio(img):
    a = 0
    for row in range(len(img)):
        for col in range(len(img[0])):
            if img[row][col]==True:
                a = a+1
    total = img.shape[0] * img.shape[1]
    return a/total

In [7]:
def Centroid(img):
    numOfWhites = 0
    a = np.array([0,0])
    for row in range(len(img)):
        for col in range(len(img[0])):
            if img[row][col]==True:
                b = np.array([row,col])
                a = np.add(a,b)
                numOfWhites += 1
    rowcols = np.array([img.shape[0], img.shape[1]])
    centroid = a/numOfWhites
    centroid = centroid/rowcols
    return centroid[0], centroid[1]

In [8]:
def EccentricitySolidity(img):
    r = regionprops(img.astype("int8"))
    return r[0].eccentricity, r[0].solidity

In [9]:
def SkewKurtosis(img):
    h,w = img.shape
    x = range(w)  # cols value
    y = range(h)  # rows value
    #calculate projections along the x and y axes
    xp = np.sum(img,axis=0)
    yp = np.sum(img,axis=1)
    #centroid
    cx = np.sum(x*xp)/np.sum(xp)
    cy = np.sum(y*yp)/np.sum(yp)
    #standard deviation
    x2 = (x-cx)**2
    y2 = (y-cy)**2
    sx = np.sqrt(np.sum(x2*xp)/np.sum(img))
    sy = np.sqrt(np.sum(y2*yp)/np.sum(img))
    
    #skewness
    x3 = (x-cx)**3
    y3 = (y-cy)**3
    skewx = np.sum(xp*x3)/(np.sum(img) * sx**3)
    skewy = np.sum(yp*y3)/(np.sum(img) * sy**3)

    #Kurtosis
    x4 = (x-cx)**4
    y4 = (y-cy)**4
    # 3 is subtracted to calculate relative to the normal distribution
    kurtx = np.sum(xp*x4)/(np.sum(img) * sx**4) - 3
    kurty = np.sum(yp*y4)/(np.sum(img) * sy**4) - 3

    return (skewx , skewy), (kurtx, kurty)

In [10]:
def getFeatures(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    img = preproc(path, display=display)
    ratio = Ratio(img)
    centroid = Centroid(img)
    eccentricity, solidity = EccentricitySolidity(img)
    skewness, kurtosis = SkewKurtosis(img)
    retVal = (ratio, centroid, eccentricity, solidity, skewness, kurtosis)
    return retVal

In [11]:
def getCSVFeatures(path, img=None, display=False):
    if img is None:
        img = mpimg.imread(path)
    temp = getFeatures(path, display=display)
    features = (temp[0], temp[1][0], temp[1][1], temp[2], temp[3], temp[4][0], temp[4][1], temp[5][0], temp[5][1])
    return features

## Saving the features

In [12]:
def makeCSV():
    if not(os.path.exists('D:\\ML_Practice\\Signature-Forgery-Detection\\Features')):
        os.mkdir('D:\\ML_Practice\\Signature-Forgery-Detection\\Features')
        print('New folder "Features" created')
    if not(os.path.exists('D:\\ML_Practice\\Signature-Forgery-Detection\\Features/Training')):
        os.mkdir('D:\\ML_Practice\\Signature-Forgery-Detection\\Features/Training')
        print('New folder "Features/Training" created')
    if not(os.path.exists('D:\\ML_Practice\\Signature-Forgery-Detection\\Features/Testing')):
        os.mkdir('D:\\ML_Practice\\Signature-Forgery-Detection\\Features/Testing')
        print('New folder "Features/Testing" created')
    # genuine signatures path
    gpath = genuine_image_paths
    # forged signatures path
    fpath = forged_image_paths
    for person in range(1,13):
        per = ('00'+str(person))[-3:]
        print('Saving features for person id-',per)
        
        with open('D:\\ML_Practice\\Signature-Forgery-Detection\\Features\\Training/training_'+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            # Training set
            for i in range(0,3):
                source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',1\n')
            for i in range(0,3):
                source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',0\n')
        
        with open('D:\\ML_Practice\\Signature-Forgery-Detection\\Features\\Testing/testing_'+per+'.csv', 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            # Testing set
            for i in range(3, 5):
                source = os.path.join(gpath, per+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',1\n')
            for i in range(3,5):
                source = os.path.join(fpath, '021'+per+'_00'+str(i)+'.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features))+',0\n')

In [13]:
makeCSV()

Saving features for person id- 001
Saving features for person id- 002
Saving features for person id- 003
Saving features for person id- 004
Saving features for person id- 005
Saving features for person id- 006
Saving features for person id- 007
Saving features for person id- 008
Saving features for person id- 009
Saving features for person id- 010
Saving features for person id- 011
Saving features for person id- 012


# TF Model 

In [14]:
def testing(path):
    feature = getCSVFeatures(path)
    if not(os.path.exists('D:\\ML_Practice\\Signature-Forgery-Detection/TestFeatures')):
        os.mkdir('D:\\ML_Practice\\Signature-Forgery-Detection/TestFeatures')
    with open('D:\\ML_Practice\\Signature-Forgery-Detection\\TestFeatures/testcsv.csv', 'w') as handle:
        handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y\n')
        handle.write(','.join(map(str, feature))+'\n')

In [15]:
n_input = 9
train_person_id = input("Enter person's id : ")
test_image_path = input("Enter path of signature image : ")
train_path = 'D:\\ML_Practice\\Signature-Forgery-Detection\\Features\\Training/training_'+train_person_id+'.csv'
testing(test_image_path)
test_path = 'TestFeatures/testcsv.csv'

def readCSV(train_path, test_path, type2=False):
    # Reading train data
    df = pd.read_csv(train_path, usecols=range(n_input))
    train_input = np.array(df.values)
    train_input = train_input.astype(np.float32, copy=False)  # Converting input to float_32
    df = pd.read_csv(train_path, usecols=(n_input,))
    temp = [elem[0] for elem in df.values]
    correct = np.array(temp)
    corr_train = keras.utils.to_categorical(correct,2)      # Converting to one hot
    # Reading test data
    df = pd.read_csv(test_path, usecols=range(n_input))
    test_input = np.array(df.values)
    test_input = test_input.astype(np.float32, copy=False)
    if not(type2):
        df = pd.read_csv(test_path, usecols=(n_input,))
        temp = [elem[0] for elem in df.values]
        correct = np.array(temp)
        corr_test = kearas.utils.to_categorical(correct,2)      # Converting to one hot
    if not(type2):
        return train_input, corr_train, test_input, corr_test
    else:
        return train_input, corr_train, test_input

ops.reset_default_graph()
# Parameters
learning_rate = 0.001
training_epochs = 1000
display_step = 1

# Network Parameters
n_hidden_1 = 7 # 1st layer number of neurons
n_hidden_2 = 10 # 2nd layer number of neurons
n_hidden_3 = 30 # 3rd layer
n_classes = 2 # no. of classes (genuine or forged)

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], seed=1)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes], seed=2))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1], seed=3)),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b3': tf.Variable(tf.random_normal([n_hidden_3])),
    'out': tf.Variable(tf.random_normal([n_classes], seed=4))
}


# Create model
def multilayer_perceptron(x):
    layer_1 = tf.tanh((tf.matmul(x, weights['h1']) + biases['b1']))
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    out_layer = tf.tanh(tf.matmul(layer_1, weights['out']) + biases['out'])
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

# Define loss and optimizer

loss_op = tf.reduce_mean(tf.squared_difference(logits, Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# For accuracies
pred = tf.nn.softmax(logits)  # Apply softmax to logits
correct_prediction = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Initializing the variables
init = tf.global_variables_initializer()

def evaluate(train_path, test_path, type2=False):   
    if not(type2):
        train_input, corr_train, test_input, corr_test = readCSV(train_path, test_path)
    else:
        train_input, corr_train, test_input = readCSV(train_path, test_path, type2)
    ans = 'Random'
    with tf.Session() as sess:
        sess.run(init)
        # Training cycle
        for epoch in range(training_epochs):
            # Run optimization op (backprop) and cost op (to get loss value)
            _, cost = sess.run([train_op, loss_op], feed_dict={X: train_input, Y: corr_train})
            if cost<0.0001:
                break
#             # Display logs per epoch step
#             if epoch % 999 == 0:
#                 print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(cost))
#         print("Optimization Finished!")
        
        # Finding accuracies
        accuracy1 =  accuracy.eval({X: train_input, Y: corr_train})
#         print("Accuracy for train:", accuracy1)
#         print("Accuracy for test:", accuracy2)
        if type2 is False:
            accuracy2 =  accuracy.eval({X: test_input, Y: corr_test})
            return accuracy1, accuracy2
        else:
            prediction = pred.eval({X: test_input})
            if prediction[0][1]>prediction[0][0]:
                print('Genuine Image')
                return True
            else:
                print('Forged Image')
                return False


def trainAndTest(rate=0.001, epochs=1700, neurons=7, display=False):    
    start = time()

    # Parameters
    global training_rate, training_epochs, n_hidden_1
    learning_rate = rate
    training_epochs = epochs

    # Network Parameters
    n_hidden_1 = neurons # 1st layer number of neurons
    n_hidden_2 = 7 # 2nd layer number of neurons
    n_hidden_3 = 30 # 3rd layer

    train_avg, test_avg = 0, 0
    n = 10
    for i in range(1,n+1):
        if display:
            print("Running for Person id",i)
        temp = ('0'+str(i))[-2:]
        train_score, test_score = evaluate(train_path.replace('01',temp), test_path.replace('01',temp))
        train_avg += train_score
        test_avg += test_score
    if display:
#         print("Number of neurons in Hidden layer-", n_hidden_1)
        print("Training average-", train_avg/n)
        print("Testing average-", test_avg/n)
        print("Time taken-", time()-start)
    return train_avg/n, test_avg/n, (time()-start)/n


evaluate(train_path, test_path, type2=True)

Enter person's id :  009
Enter path of signature image :  D:\ML_Practice\Signature-Forgery-Detection\forged\021004_001.png



Forged Image


False

In [16]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import pickle
from time import time

# Ensure TensorFlow uses a compatible version
tf.compat.v1.disable_eager_execution()
ops = tf.compat.v1

# Define number of inputs
n_input = 9

# Function to create necessary directories
def makeCSV():
    base_dir = 'D:\\ML_Practice\\Signature-Forgery-Detection\\Features'
    
    for sub_dir in ["", "Training", "Testing"]:
        full_path = os.path.join(base_dir, sub_dir)
        if not os.path.exists(full_path):
            os.mkdir(full_path)
            print(f'New folder "{full_path}" created')

    gpath = genuine_image_paths  # Define these paths correctly
    fpath = forged_image_paths  

    for person in range(1, 13):
        per = f"{person:03d}"
        print(f'Saving features for person id - {per}')
        
        train_file = os.path.join(base_dir, 'Training', f'training_{per}.csv')
        test_file = os.path.join(base_dir, 'Testing', f'testing_{per}.csv')

        with open(train_file, 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            for i in range(0, 3):
                source = os.path.join(gpath, f'{per}{per}_00{i}.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features)) + ',1\n')

            for i in range(0, 3):
                source = os.path.join(fpath, f'021{per}_00{i}.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features)) + ',0\n')

        with open(test_file, 'w') as handle:
            handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
            for i in range(3, 5):
                source = os.path.join(gpath, f'{per}{per}_00{i}.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features)) + ',1\n')

            for i in range(3, 5):
                source = os.path.join(fpath, f'021{per}_00{i}.png')
                features = getCSVFeatures(path=source)
                handle.write(','.join(map(str, features)) + ',0\n')

# Execute CSV creation
makeCSV()

# Function to read CSV files correctly
def readCSV(train_path, test_path, type2=False):
    try:
        df_train = pd.read_csv(train_path)
        if df_train.shape[1] < n_input + 1:
            raise ValueError(f"Expected at least {n_input + 1} columns in {train_path}, but found {df_train.shape[1]}")

        train_input = np.array(df_train.iloc[:, :n_input].values, dtype=np.float32)
        correct_train = np.array(df_train.iloc[:, n_input].values, dtype=np.int32)
        corr_train = keras.utils.to_categorical(correct_train, 2)

        df_test = pd.read_csv(test_path)
        if df_test.shape[1] < n_input:
            raise ValueError(f"Expected at least {n_input} columns in {test_path}, but found {df_test.shape[1]}")

        test_input = np.array(df_test.iloc[:, :n_input].values, dtype=np.float32)

        if not type2:
            correct_test = np.array(df_test.iloc[:, n_input].values, dtype=np.int32)
            corr_test = keras.utils.to_categorical(correct_test, 2)
            return train_input, corr_train, test_input, corr_test
        else:
            return train_input, corr_train, test_input

    except Exception as e:
        print("Error reading CSV:", e)
        return None, None, None, None

# Neural Network Parameters
learning_rate = 0.001
training_epochs = 1000
n_hidden_1 = 7
n_hidden_2 = 10
n_hidden_3 = 30
n_classes = 2

# TensorFlow placeholders
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_classes])

# Network weights and biases
weights = {
    'h1': tf.Variable(tf.random.normal([n_input, n_hidden_1], seed=1)),
    'h2': tf.Variable(tf.random.normal([n_hidden_1, n_hidden_2])),
    'h3': tf.Variable(tf.random.normal([n_hidden_2, n_hidden_3])),
    'out': tf.Variable(tf.random.normal([n_hidden_1, n_classes], seed=2))
}

biases = {
    'b1': tf.Variable(tf.random.normal([n_hidden_1], seed=3)),
    'b2': tf.Variable(tf.random.normal([n_hidden_2])),
    'b3': tf.Variable(tf.random.normal([n_hidden_3])),
    'out': tf.Variable(tf.random.normal([n_classes], seed=4))
}

# MLP Model
def multilayer_perceptron(x):
    layer_1 = tf.nn.tanh(tf.matmul(x, weights['h1']) + biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_3 = tf.add(tf.matmul(layer_2, weights['h3']), biases['b3'])
    out_layer = tf.nn.tanh(tf.matmul(layer_1, weights['out']) + biases['out'])
    return out_layer

# Define loss and optimizer
logits = multilayer_perceptron(X)
#loss_op = tf.reduce_mean(tf.squared_difference(logits, Y))
loss_op = tf.reduce_mean(tf.math.squared_difference(logits, Y))
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Accuracy computation
pred = tf.nn.softmax(logits)
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initialize variables
init = tf.compat.v1.global_variables_initializer()

# Training Function
def train_and_save_model(train_path, test_path, model_filename='signature_model.pkl'):
    train_input, corr_train, test_input, corr_test = readCSV(train_path, test_path)

    with tf.compat.v1.Session() as sess:
        sess.run(init)

        for epoch in range(training_epochs):
            _, cost = sess.run([train_op, loss_op], feed_dict={X: train_input, Y: corr_train})
            if cost < 0.0001:
                break

        print("Training Complete!")
        
        # Save model to pickle
        model_data = {
            'weights': sess.run(weights),
            'biases': sess.run(biases)
        }
        with open(model_filename, 'wb') as file:
            pickle.dump(model_data, file)
        print(f"Model saved to {model_filename}")

# Execute training and save model
train_path = 'D:\\ML_Practice\\Signature-Forgery-Detection\\Features\\Training/training_001.csv'
test_path = 'D:\\ML_Practice\\Signature-Forgery-Detection\\Features\\Testing/testing_001.csv'
train_and_save_model(train_path, test_path)


Saving features for person id - 001
Saving features for person id - 002
Saving features for person id - 003
Saving features for person id - 004
Saving features for person id - 005
Saving features for person id - 006
Saving features for person id - 007
Saving features for person id - 008
Saving features for person id - 009
Saving features for person id - 010
Saving features for person id - 011
Saving features for person id - 012
Training Complete!
Model saved to signature_model.pkl


## 📌 CELL 1: Import Libraries

In [ ]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# import os
# import joblib  # For saving and loading model
# from tensorflow import keras
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Input


## 📌 CELL 2: Feature Extraction Function

In [ ]:
# def getCSVFeatures(path):
#     # Dummy function to extract features from an image (Replace with actual feature extraction)
#     # This function should return an array of 9 extracted features
#     return np.random.rand(9)  # Replace with real feature extraction

## 📌 CELL 3: Create CSV Files for Training & Testing

In [ ]:
# import os

# def makeCSV():
#     base_path = 'D:\\ML_Practice\\Signature-Forgery-Detection\\Features1'
    
#     # Create directories if they don't exist
#     for folder in ['Features1', 'Features1/Training', 'Features1/Testing']:
#         path = os.path.join('D:\\ML_Practice\\Signature-Forgery-Detection', folder)
#         if not os.path.exists(path):
#             os.mkdir(path)
#             print(f'📁 Created new folder: "{folder}"')

#     # Paths for genuine and forged images
#     gpath = 'D:\\ML_Practice\\Signature-Forgery-Detection\\genuine'
#     fpath = 'D:\\ML_Practice\\Signature-Forgery-Detection\\forged'

#     print("\n🔄 Starting feature extraction and saving CSV files...\n")

#     for person in range(1, 13):  # Loop for persons 001 to 012
#         per = f'{person:03d}'  # Format person ID (001, 002, etc.)
#         print(f'✅ Saving features for person id - {per}')

#         # Training CSV
#         train_file = f'{base_path}/Training/training_{per}.csv'
#         with open(train_file, 'w') as handle:
#             handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
#             for i in range(3):  # 3 genuine + 3 forged
#                 handle.write(','.join(map(str, getCSVFeatures(f'{gpath}/{per}{per}_00{i}.png'))) + ',1\n')
#                 handle.write(','.join(map(str, getCSVFeatures(f'{fpath}/021{per}_00{i}.png'))) + ',0\n')

#         # Testing CSV
#         test_file = f'{base_path}/Testing/testing_{per}.csv'
#         with open(test_file, 'w') as handle:
#             handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y,output\n')
#             for i in range(3, 5):  # 2 genuine + 2 forged
#                 handle.write(','.join(map(str, getCSVFeatures(f'{gpath}/{per}{per}_00{i}.png'))) + ',1\n')
#                 handle.write(','.join(map(str, getCSVFeatures(f'{fpath}/021{per}_00{i}.png'))) + ',0\n')

#     print("\n✅ Feature extraction complete! All CSV files saved successfully. 🎯")

# # Run the function
# makeCSV()


## 📌 CELL 5: Testing Function

In [ ]:
# def testing(path):
#     feature = getCSVFeatures(path)

#     # Create folder if not exists
#     test_folder = 'D:\\ML_Practice\\Signature-Forgery-Detection\\TestFeatures1'
#     if not os.path.exists(test_folder):
#         os.mkdir(test_folder)

#     # Save extracted features
#     with open(f'{test_folder}/testcsv.csv', 'w') as handle:
#         handle.write('ratio,cent_y,cent_x,eccentricity,solidity,skew_x,skew_y,kurt_x,kurt_y\n')
#         handle.write(','.join(map(str, feature)) + '\n')


## 📌 CELL 6: Read CSV and Prepare Data

In [ ]:
# # import pandas as pd
# # import numpy as np
# # from tensorflow.keras.utils import to_categorical

# # n_input = 9  # Number of feature columns

# # def readCSV(train_path, test_path, type2=False):
# #     # Check the actual number of columns in the CSV before reading
# #     df_sample = pd.read_csv(train_path, nrows=1)  # Read only the first row
# #     actual_columns = df_sample.shape[1]  # Get actual column count

# #     if actual_columns < n_input + 1:  # +1 for the label column
# #         raise ValueError(f"Expected at least {n_input + 1} columns, but found only {actual_columns} in {train_path}")

# #     # Read feature columns
# #     df_train = pd.read_csv(train_path, usecols=range(n_input), header=0)
# #     train_input = np.array(df_train.values, dtype=np.float32)

# #     # Read label column
# #     df_labels = pd.read_csv(train_path, usecols=[n_input], header=0)
# #     correct = np.array(df_labels.values).flatten()
# #     corr_train = to_categorical(correct, num_classes=2)  # Convert labels to one-hot encoding

# #     # Read test data
# #     df_test = pd.read_csv(test_path, usecols=range(n_input), header=None)
# #     test_input = np.array(df_test.values, dtype=np.float32)

# #     if not type2:
# #         df_test_labels = pd.read_csv(test_path, usecols=[n_input], header=None)
# #         correct = np.array(df_test_labels.values).flatten()
# #         corr_test = to_categorical(correct, num_classes=2)
# #         return train_input, corr_train, test_input, corr_test

# #     return train_input, corr_train, test_input



# import pandas as pd
# import numpy as np
# from tensorflow.keras.utils import to_categorical

# n_input = 9  # Number of feature columns

# def readCSV(train_path, test_path, type2=False):
#     # Read training data
#     df_train = pd.read_csv(train_path, header=0)  
#     df_train = df_train.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric to NaN

#     # Extract features and labels
#     train_input = np.array(df_train.iloc[:, :n_input], dtype=np.float32)
#     correct = np.array(df_train.iloc[:, n_input], dtype=np.int32)
#     corr_train = to_categorical(correct, num_classes=2)

#     # Read test data
#     df_test = pd.read_csv(test_path, header=0)
#     df_test = df_test.apply(pd.to_numeric, errors='coerce')
#     test_input = np.array(df_test.iloc[:, :n_input], dtype=np.float32)

#     # Handle test labels properly
#     if not type2:
#         correct = np.array(df_test.iloc[:, n_input], dtype=np.int32)
#         corr_test = to_categorical(correct, num_classes=2)
#     else:
#         corr_test = None  # Return None if type2=True to avoid unpacking errors

#     return train_input, corr_train, test_input, corr_test  # Always return 4 values


## 📌 CELL 7: Define Neural Network

In [ ]:
# import tensorflow as tf

# # Parameters
# learning_rate = 0.001
# training_epochs = 1000
# n_hidden_1 = 7
# n_hidden_2 = 10
# n_hidden_3 = 30
# n_classes = 2
# n_input = 9  # Define input feature size

# # Define Model Using Functional API
# inputs = tf.keras.Input(shape=(n_input,))
# x = tf.keras.layers.Dense(n_hidden_1, activation='tanh')(inputs)
# x = tf.keras.layers.Dense(n_hidden_2, activation='relu')(x)
# x = tf.keras.layers.Dense(n_hidden_3, activation='relu')(x)
# outputs = tf.keras.layers.Dense(n_classes, activation='tanh')(x)

# # Build Model
# mlp_model = tf.keras.Model(inputs=inputs, outputs=outputs)

# # Show Model Summary
# mlp_model.summary()

# # Compile Model (Adding Loss & Optimizer)
# mlp_model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#     loss=tf.keras.losses.MeanSquaredError(),
#     metrics=["accuracy"]
# )


## 📌 CELL 8: Model Evaluation

In [ ]:
# import tensorflow as tf

# def evaluate(train_path, test_path, type2=False):
#     train_input, corr_train, test_input, corr_test = readCSV(train_path, test_path) if not type2 else readCSV(train_path, test_path, type2)

#     init = tf.compat.v1.global_variables_initializer()  # Initialize variables

#     with tf.compat.v1.Session() as sess:
#         sess.run(init)
#         for epoch in range(training_epochs):
#             _, cost = sess.run([train_op, loss_op], feed_dict={X: train_input, Y: corr_train})

#         if type2:
#             prediction = sess.run(pred_op, feed_dict={X: test_input})
#             return prediction.argmax(axis=1) == 1  # Assuming 1 = Genuine, 0 = Forged
#         else:
#             accuracy = sess.run(accuracy_op, feed_dict={X: test_input, Y: corr_test})
#             return accuracy


## 📌 CELL 9: Prediction Function

In [ ]:
# def predict_signature():
#     person_id = input("Enter person's ID: ")
#     test_image_path = input("Enter the path of the signature image: ")

#     train_path = f'D:\\ML_Practice\\Signature-Forgery-Detection\\Features1\\Training/training_{person_id}.csv'
#     testing(test_image_path)
#     test_path = 'D:\\ML_Practice\\Signature-Forgery-Detection\\TestFeatures1/testcsv.csv'

#     accuracy_test = evaluate(train_path, test_path, type2=True)
#     print("\n✅ Prediction: The signature is **Genuine**" if accuracy_test else "\n❌ Prediction: The signature is **Forged**")


## 📌 CELL 10: Run Prediction

In [ ]:
# predict_signature()

## 📌 CELL: Save Model as .pkl File for Deployment

In [ ]:
# import joblib
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense

# # Load training data
# train_person_id = "001"  # Change this based on person ID
# train_path = f'D:\\ML_Practice\\Signature-Forgery-Detection\\Features\\Training/training_{train_person_id}.csv'

# # Read CSV
# df = pd.read_csv(train_path)
# X_train = df.iloc[:, :-1].values  # All columns except last (features)
# y_train = df.iloc[:, -1].values   # Last column (labels)

# # Convert labels to categorical (one-hot encoding)
# y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)

# # Define Model
# model = Sequential([
#     Dense(7, activation='tanh', input_shape=(X_train.shape[1],)),
#     Dense(10, activation='relu'),
#     Dense(30, activation='relu'),
#     Dense(2, activation='softmax')  # Output layer with 2 classes
# ])

# # Compile Model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train Model
# model.fit(X_train, y_train, epochs=100, verbose=1)

# # Save Model as .pkl
# pkl_filename = "D:\\ML_Practice\\Signature-Forgery-Detection\\model.pkl"
# joblib.dump(model, pkl_filename)

# print(f"✅ Model saved successfully at: {pkl_filename}")
